In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mbertav3base/spm.model
/kaggle/input/mbertav3base/config.json
/kaggle/input/mbertav3base/training_args.bin
/kaggle/input/mbertav3base/tokenizer_config.json
/kaggle/input/mbertav3base/pytorch_model.bin
/kaggle/input/mbertav3base/special_tokens_map.json
/kaggle/input/mbertav3base/added_tokens.json
/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv
/kaggle/input/cpc-codes/titles.csv
/kaggle/input/debertav3small/spm.model
/kaggle/input/debertav3small/config.json
/kaggle/input/debertav3small/README.md
/kaggle/input/debertav3small/tf_model.h5
/kaggle/input/debertav3small/tokenizer_config.json
/kaggle/input/debertav3small/pytorch_model.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/training_args.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer.json
/kaggle/input/aigl-patentsberta-ka

# Credits:
this notebook is based on the notbook ["Getting started with NLP for absolute beginners"](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners) by @Jeremy Howard

## Imports

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns
import matplotlib.pyplot as plt

# EDA

In [3]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
print(train.shape)
train.head()

(36473, 5)


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
train.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
train.groupby('score').count()

,id,anchor,target,context
score,,,,
0.00,7471,7471,7471,7471
0.25,11519,11519,11519,11519
0.50,12300,12300,12300,12300
0.75,4029,4029,4029,4029
1.00,1154,1154,1154,1154


## Score meanings according to [Data Description](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data?select=train.csv)

The scores are in the 0-1 range with increments of 0.25 with the following meanings:

    1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
    0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
    0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
    0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
    0.0 - Unrelated.


# Test Corpus

In [6]:
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")

In [7]:
test.shape

(36, 4)

In [8]:
def cosine(u, v):
    """
    cosine similarity definition
    """
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# External Context

In [9]:
# ajouter le contexte https://www.cooperativepatentclassification.org/Archive
# extraire le contexte du fichier xml

In [10]:
titles = pd.read_csv("../input/cpc-codes/titles.csv")
print(titles.shape)
titles.head()

(260476, 7)


,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


# HuggingFace AutoModelForSequenceClassification

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr # pour trouver les correspondances entre les résultats sur des échelles variées
from sklearn.model_selection import StratifiedGroupKFold

In [12]:
import transformers
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

import torch
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [13]:
import warnings
import logging
from IPython.display import display, HTML

# Corpora merge

In [14]:
train['section'] = train["context"].str[0]
print(train.shape)
display(train.head())
titles=titles.rename(columns={"code": "context"})
display(titles.head())
train = train.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
train = train.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(train.head())

(36473, 6)


,id,anchor,target,context,score,section
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,score,section,context_title,section_title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES


In [15]:
logging.disable(logging.WARNING)
warnings.simplefilter('ignore')

# PatentSBerta

In [16]:
#model_name = 'AI-Growth-Lab/PatentSBERTa_Kaggle_V3' #(online version)
model_name = '../input/aigl-patentsberta-kaggle-v3'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
tokenizer.all_special_tokens

['<s>', '</s>', '[UNK]', '<pad>', '<mask>']

In [19]:
def clean(x):
    t = x.lower()
    t = t.replace("[",'')
    t = t.replace(";",'')
    t = t.replace(",",'')
    t = t.replace("]",'')
    t = t.replace(":",'')
    t = t.replace("(",'')
    t = t.replace(")",'')
    t = t.replace("{",'')
    t = t.replace("}",'')
    t = t.replace("/",' ')
    t = t.replace("-",' ')
    return t

In [20]:
train["inputs"] = train["section_title"].apply(clean) + " [SEP] " + train["anchor"] + " [SEP] " + train["target"]

In [21]:
def tok_func(x):
    return tokenizer(x["inputs"])

In [22]:
train_ds = Dataset.from_pandas(train).rename_column('score', 'label')

In [23]:
inps = "anchor","target","context","context_title","section_title"
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [24]:
tok_ds

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [25]:
# bs - batch size
lr,bs = 8e-5,64
# wd: weight decay
wd,epochs = 0.01,4

In [26]:
anchors = train["anchor"].unique()
np.random.shuffle(anchors)

In [27]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [28]:
is_val = np.isin(train.anchor, val_anchors)
idxs = np.arange(len(train))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

(9574, 26899)

In [29]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [30]:
def corr(eval_pred): 
    logits, labels = eval_pred
    logits = logits.reshape(-1)
    return {'pearson': np.corrcoef(logits, labels)[0][1]}

In [31]:
#args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    #evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
     #num_train_epochs=epochs, weight_decay=wd, report_to='none')
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr)

In [33]:
dds['train']

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 26899
})

In [34]:
eval = trainer.evaluate()

In [35]:
eval

{'eval_loss': 0.011825671419501305,
 'eval_pearson': 0.9191270175669126,
 'eval_runtime': 9.1621,
 'eval_samples_per_second': 1044.96,
 'eval_steps_per_second': 8.186}

In [36]:
trainer.num_examples(dds['train'])

1721536

In [37]:
#trainer.save_model()

# Test corpus preprocessing

In [38]:
test.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [39]:
test['section'] = test["context"].str[0]
print(test.shape)
display(titles.head())
test = test.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
test = test.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(test.head())

(36, 5)


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,section,context_title,section_title
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G,OPTICS,PHYSICS
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G,OPTICS,PHYSICS
2,7aa5908a77a7ec24,el display,illumination,G02,G,OPTICS,PHYSICS
3,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11,G,INFORMATION STORAGE,PHYSICS
4,12ca31f018a2e2b9,speed control means,control loop,G05,G,CONTROLLING; REGULATING,PHYSICS


In [40]:
test["inputs"] = test["section_title"].apply(clean) + " [SEP] " + test["anchor"] + " [SEP] " + test["target"]

In [41]:
test_ds = Dataset.from_pandas(test)

In [42]:
test_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'section', 'context_title', 'section_title', 'inputs', '__index_level_0__'],
    num_rows: 36
})

In [43]:
inps = "anchor","target","context","context_title","section_title"
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [44]:
preds_sberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_sberta

array([[ 0.48749384],
       [ 0.948901  ],
       [ 0.28694433],
       [ 0.50278115],
       [ 0.31292188],
       [ 0.73349011],
       [ 0.4781965 ],
       [ 0.79074466],
       [ 1.05603766],
       [ 0.2435392 ],
       [ 0.24521858],
       [ 0.55106753],
       [-0.01621631],
       [ 0.5375756 ],
       [ 0.63313073],
       [-0.00538462],
       [ 0.17454851],
       [ 0.79013216],
       [ 0.79604405],
       [-0.00391432],
       [ 0.27687433],
       [ 0.27295172],
       [ 0.52643836],
       [ 0.3617987 ],
       [ 0.01224874],
       [ 0.30376944],
       [ 0.29838696],
       [ 0.53494966],
       [ 0.12322883],
       [ 0.49000978],
       [ 0.48843858],
       [ 0.14339146],
       [-0.01222257],
       [ 0.1442931 ],
       [-0.0065867 ],
       [-0.00724098]])

In [45]:
type(preds_sberta)

numpy.ndarray

In [46]:
del model_name, tokenizer, model, tok_ds, test_tok_ds, dds

# Deberta-V3_small inference

In [47]:
#model_name = 'microsoft/deberta-v3-small' (online version)
model_name = '../input/debertav3small'

In [48]:
def corr_deberta(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [49]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [50]:
tokenizer.all_special_tokens

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']

In [51]:
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [52]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr_deberta)

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028442,0.777731
2,0.045600,0.025528,0.796421
3,0.019000,0.024905,0.804414
4,0.012200,0.025109,0.806257


TrainOutput(global_step=1684, training_loss=0.023978924878702595, metrics={'train_runtime': 273.8329, 'train_samples_per_second': 392.926, 'train_steps_per_second': 6.15, 'total_flos': 466030408777770.0, 'train_loss': 0.023978924878702595, 'epoch': 4.0})

In [55]:
eval_deberta = trainer.evaluate()
eval_deberta

{'eval_loss': 0.025108594447374344,
 'eval_pearson': 0.8062570804201469,
 'eval_runtime': 6.3696,
 'eval_samples_per_second': 1503.072,
 'eval_steps_per_second': 11.775,
 'epoch': 4.0}

In [56]:
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [57]:
preds_deberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_deberta

array([[ 0.4294847 ],
       [ 0.67459488],
       [ 0.37065387],
       [ 0.44099724],
       [ 0.12224401],
       [ 0.68543702],
       [ 0.54469669],
       [ 0.76059484],
       [ 1.07091916],
       [ 0.21218741],
       [ 0.27439523],
       [ 0.57111704],
       [-0.01673831],
       [ 0.60749382],
       [ 0.6025002 ],
       [-0.01589945],
       [ 0.27302894],
       [ 0.81717235],
       [ 0.79191828],
       [-0.0060753 ],
       [ 0.26814482],
       [ 0.26163366],
       [-0.03014177],
       [ 0.22790876],
       [-0.0024351 ],
       [ 0.33818492],
       [ 0.42688107],
       [ 0.50269574],
       [ 0.24224597],
       [ 0.53079408],
       [ 0.48534891],
       [ 0.25089929],
       [-0.02091357],
       [ 0.24875912],
       [-0.01795117],
       [-0.02282841]])

In [58]:
del model_name, tokenizer, model, tok_ds, test_tok_ds, dds

# mdeberta-v3-base

In [59]:
model_name = '../input/mbertav3base'

In [60]:
def corr_deberta(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [61]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [62]:
tokenizer.all_special_tokens

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']

In [63]:
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [64]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [65]:
eval_mberta = trainer.evaluate()
eval_mberta

{'eval_loss': 0.025108594447374344,
 'eval_pearson': 0.8062570804201469,
 'eval_runtime': 6.4153,
 'eval_samples_per_second': 1492.365,
 'eval_steps_per_second': 11.691,
 'epoch': 4.0}

In [66]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr_deberta)

In [67]:
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [68]:
preds_mberta = trainer.predict(test_tok_ds).predictions.astype(float)
preds_mberta

array([[ 0.57342774],
       [ 0.88247854],
       [ 0.344686  ],
       [ 0.47689673],
       [ 0.10138647],
       [ 0.76198739],
       [ 0.51528311],
       [ 0.78051585],
       [ 1.05626392],
       [ 0.19956696],
       [ 0.25408095],
       [ 0.53530359],
       [-0.0234694 ],
       [ 0.48995924],
       [ 0.75317681],
       [-0.01808831],
       [ 0.27053031],
       [ 0.79927522],
       [ 0.76635557],
       [-0.01418376],
       [ 0.36837041],
       [ 0.27161333],
       [-0.01907934],
       [ 0.26742202],
       [-0.01893202],
       [ 0.2128399 ],
       [ 0.27632561],
       [ 0.52651244],
       [ 0.23279697],
       [ 0.50571138],
       [ 0.33413237],
       [ 0.2365689 ],
       [-0.01838638],
       [ 0.27539331],
       [-0.01062036],
       [-0.00368166]])

# Ensemble

ensemble methode inspired by Valentin Werner's "In Depth EDA & 3 Model Ensemble" notebook 

In [69]:
test_preds = 0.333 * preds_sberta + (0.333) * preds_deberta + (0.333) * preds_mberta

In [70]:
test_preds

array([[ 0.49630529],
       [ 0.83448948],
       [ 0.33376064],
       [ 0.47308482],
       [ 0.17867193],
       [ 0.72624454],
       [ 0.51221271],
       [ 0.77650783],
       [ 1.06001251],
       [ 0.21821276],
       [ 0.25764035],
       [ 0.55194356],
       [-0.0187892 ],
       [ 0.54446454],
       [ 0.66227298],
       [-0.013111  ],
       [ 0.23912988],
       [ 0.80139105],
       [ 0.78398786],
       [-0.00804974],
       [ 0.30415872],
       [ 0.26846417],
       [ 0.15891334],
       [ 0.28542412],
       [-0.00303642],
       [ 0.28464649],
       [ 0.33353069],
       [ 0.52086456],
       [ 0.1992245 ],
       [ 0.50832958],
       [ 0.43553731],
       [ 0.21007626],
       [-0.017157  ],
       [ 0.22259236],
       [-0.01170769],
       [-0.0112391 ]])

In [71]:
df_submission = pd.DataFrame()

In [72]:
df_submission['id'] = test['id'].copy()

In [73]:
df_submission

,id
0,4112d61851461f60
1,5203a36c501f1b7c
2,7aa5908a77a7ec24
3,442c114ed5c4e3c9
4,12ca31f018a2e2b9
5,9f2279ce667b21dc
6,b9ea2b06a878df6f
7,09e418c93a776564
8,ae0262c02566d2ce
9,03ba802ed4029e4d


In [74]:
preds_list = test_preds.tolist()

In [75]:
preds_list

[[0.4963052895069122],
 [0.8344894791841507],
 [0.33376063826680186],
 [0.4730848178565502],
 [0.17867193411290647],
 [0.7262445355653764],
 [0.5122127072811127],
 [0.7765078337788582],
 [1.0600125077962876],
 [0.21821275846660138],
 [0.25764035296440124],
 [0.5519435597062111],
 [-0.018789197992533445],
 [0.5444645438790322],
 [0.6622729766964912],
 [-0.013111004364676774],
 [0.23912988406419755],
 [0.801391051054001],
 [0.7839878618717194],
 [-0.008049736739136278],
 [0.3041587246656418],
 [0.26846417248249055],
 [0.15891334224678577],
 [0.2854241183102131],
 [-0.0030364202382043006],
 [0.2846464898586273],
 [0.3335306852459907],
 [0.5208645619153978],
 [0.19922449768334627],
 [0.5083295767307282],
 [0.43553731334209445],
 [0.21007626119256018],
 [-0.01715700238477439],
 [0.22259236067533494],
 [-0.011707688416354358],
 [-0.011239097947720439]]

In [76]:
flat_list = [x for xs in preds_list for x in xs]
flat_list

[0.4963052895069122,
 0.8344894791841507,
 0.33376063826680186,
 0.4730848178565502,
 0.17867193411290647,
 0.7262445355653764,
 0.5122127072811127,
 0.7765078337788582,
 1.0600125077962876,
 0.21821275846660138,
 0.25764035296440124,
 0.5519435597062111,
 -0.018789197992533445,
 0.5444645438790322,
 0.6622729766964912,
 -0.013111004364676774,
 0.23912988406419755,
 0.801391051054001,
 0.7839878618717194,
 -0.008049736739136278,
 0.3041587246656418,
 0.26846417248249055,
 0.15891334224678577,
 0.2854241183102131,
 -0.0030364202382043006,
 0.2846464898586273,
 0.3335306852459907,
 0.5208645619153978,
 0.19922449768334627,
 0.5083295767307282,
 0.43553731334209445,
 0.21007626119256018,
 -0.01715700238477439,
 0.22259236067533494,
 -0.011707688416354358,
 -0.011239097947720439]

In [77]:
df_submission['score' ] = flat_list

In [78]:
df_submission

,id,score
0,4112d61851461f60,0.496305
1,5203a36c501f1b7c,0.834489
2,7aa5908a77a7ec24,0.333761
3,442c114ed5c4e3c9,0.473085
4,12ca31f018a2e2b9,0.178672
5,9f2279ce667b21dc,0.726245
6,b9ea2b06a878df6f,0.512213
7,09e418c93a776564,0.776508
8,ae0262c02566d2ce,1.060013
9,03ba802ed4029e4d,0.218213


In [79]:
#submission = datasets.Dataset.from_dict({
  #  'id': test_ds['id'],
 #   'score': preds
#})

df_submission.to_csv('submission.csv', index=False)